In [9]:
from cryptography.hazmat.primitives.asymmetric import rsa, ec, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import os

### 🔹 Step 1: Receiver Generates Keys (UNCHANGED)
def generate_receiver_keys():
    """Generates RSA and ECC key pairs for the receiver."""
    private_rsa = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_rsa = private_rsa.public_key()

    private_ecc = ec.generate_private_key(ec.SECP256R1())
    public_ecc = private_ecc.public_key()

    return private_rsa, public_rsa, private_ecc, public_ecc

### 🔹 Step 2: Sender Derives AES Key with ECDH + HKDF (FIXED)
def generate_shared_secret(private_ecc_sender, public_ecc_receiver):
    """Computes ECDH shared secret and derives AES key using HKDF."""
    shared_secret = private_ecc_sender.exchange(ec.ECDH(), public_ecc_receiver)
    
    # Secure key derivation with HKDF
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'hybrid_file_encryption',
    )
    return hkdf.derive(shared_secret)

### 🔹 Step 3: Encrypt AES Key with RSA (UNCHANGED)
def encrypt_aes_key(aes_key, public_rsa_receiver):
    """Encrypts the AES key using RSA-OAEP."""
    return public_rsa_receiver.encrypt(
        aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

### 🔹 Step 4: Encrypt File with AES-GCM (UNCHANGED)
def encrypt_file(file_path, aes_key):
    """Encrypts a file using AES-GCM."""
    iv = os.urandom(12)
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(iv))
    encryptor = cipher.encryptor()

    with open(file_path, "rb") as f:
        plaintext = f.read()

    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    return iv, encryptor.tag, ciphertext

### 🔹 Step 5: Decrypt AES Key with RSA (UNCHANGED)
def decrypt_aes_key(encrypted_aes_key, private_rsa_receiver):
    """Decrypts AES key using RSA private key."""
    return private_rsa_receiver.decrypt(
        encrypted_aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

### 🔹 Step 6: Decrypt File with AES-GCM (UNCHANGED)
def decrypt_file(encrypted_data, aes_key, iv, tag):
    """Decrypts file using AES-GCM."""
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(iv, tag))
    decryptor = cipher.decryptor()
    return decryptor.update(encrypted_data) + decryptor.finalize()

### 🔹 Full Secure Workflow with FIXES
def main():
    # ➤ Receiver Setup
    private_rsa, public_rsa, private_ecc, public_ecc = generate_receiver_keys()

    # ➤ Sender Setup
    sender_ecc_private = ec.generate_private_key(ec.SECP256R1())  # Ephemeral key
    sender_ecc_public = sender_ecc_private.public_key()

    # ➤ Sender: Derive AES Key via ECDH
    aes_key_sender = generate_shared_secret(sender_ecc_private, public_ecc)

    # ➤ Sender: Encrypt AES Key with RSA
    encrypted_aes_key = encrypt_aes_key(aes_key_sender, public_rsa)

    # ➤ Sender: Prepare test file
    file_path = "test_file.txt"
    with open(file_path, "w") as f:
        f.write("Secure hybrid encryption test!")

    # ➤ Sender: Encrypt File
    iv, tag, ciphertext = encrypt_file(file_path, aes_key_sender)

    # ➤ Simulate Transmission: Send these to receiver
    transmission_data = {
        "encrypted_aes_key": encrypted_aes_key,
        "sender_ecc_public": sender_ecc_public.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ),
        "iv": iv,
        "tag": tag,
        "ciphertext": ciphertext
    }

    # ➤ Receiver: Decrypt AES Key with RSA
    decrypted_aes_key = decrypt_aes_key(
        transmission_data["encrypted_aes_key"], private_rsa
    )

    # ➤ Receiver: Re-derive AES Key via ECDH (CRITICAL FIX)
    # Deserialize sender's public key
    sender_public_ecc = serialization.load_pem_public_key(
        transmission_data["sender_ecc_public"]
    )
    
    # Recompute shared secret and AES key
    aes_key_receiver = generate_shared_secret(private_ecc, sender_public_ecc)

    # Validate keys match (MITM prevention)
    if decrypted_aes_key != aes_key_receiver:
        raise ValueError("AES Key Mismatch! Potential MITM attack detected.")

    # ➤ Receiver: Decrypt File
    decrypted_data = decrypt_file(
        transmission_data["ciphertext"],
        aes_key_receiver,
        transmission_data["iv"],
        transmission_data["tag"]
    )

    # Save decrypted file
    with open("decrypted_file.txt", "wb") as f:
        f.write(decrypted_data)

    print("✅ Secure hybrid encryption/decryption successful!")

if __name__ == "__main__":
    main()

✅ Secure hybrid encryption/decryption successful!
